## Use OAuth and MSGrapsh to read the user's profile

To create an app registration in the Azure portal for interactive device authentication and set up the necessary scopes to read the user's profile from Microsoft Graph, follow these steps:

### Using the Azure Portal

### Step 1: Sign in to the Azure Portal
1. Open a web browser and go to the [Azure Portal](https://portal.azure.com/).
2. Sign in with your Azure account credentials.

### Step 2: Navigate to Azure Active Directory
1. In the Azure portal, select **Azure Active Directory** from the left-hand navigation pane.

### Step 3: Register a New Application
1. In the Azure Active Directory pane, select **App registrations**.
2. Click on **New registration** at the top of the App registrations pane.
3. Fill in the following details:
   - **Name**: Enter a name for your application (e.g., "AntRunner").
   - **Supported account types**: Choose the account types that your application will support. Typically, for organizational use, select "Accounts in this organizational directory only".
   - **Redirect URI (optional)**: Leave this blank for now as it's not required for device code flow.
4. Click **Register** to create the app registration.

### Step 4: Configure API Permissions
1. After registration, you will be redirected to the app's **Overview** page.
2. In the left-hand menu, select **API permissions**.
3. Click on **Add a permission**.
4. Select **Microsoft Graph**.
5. Choose **Delegated permissions**.
6. In the search box, type `User.Read` and select the **User.Read** permission which allows the app to sign in and read the user's profile.
7. Click **Add permissions**.

### Step 5: Enable Public Client Flows
1. In the left-hand menu, select **Authentication**.
2. Scroll down to the **Advanced settings** section.
3. Set the **Allow public client flows** toggle to **Yes**.
4. Click **Save**.

### Step 6: Configure Platform Settings
1. In the **Authentication** section, click on **Add a platform**.
2. Choose **Mobile and desktop applications**.
3. Ensure that the **Redirect URI** is set to `http://localhost` 
4. Click **Configure**.
5. Click **Save**.

### Step 7: Note Down Application (Client) ID and Directory (Tenant) ID
1. Go back to the **Overview** section of your app.
2. Note down the **Application (client) ID** and **Directory (tenant) ID**. These values are required for authentication.

### Using Azure CLI

### Step 1: Install Azure CLI
Ensure you have the Azure CLI installed on your local machine. If not, you can download and install it from the [official Azure CLI installation page](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli).

### Step 2: Sign in to Azure
Open a terminal or command prompt and sign in to your Azure account using the following command:
```sh
az login
```
Follow the instructions in the terminal to complete the authentication process.

### Step 3: Create a New App Registration
Use the following command to create a new app registration:
```sh
az ad app create --display-name "AntRunner" --available-to-other-tenants false --sign-in-audience "AzureADMyOrg"
```
This command creates an app registration with the specified display name and sets the sign-in audience to your organization.

### Step 4: Assign API Permissions
Next, you'll need to assign the `User.Read` permission to your app registration. First, get the app ID of your newly created app:
```sh
az ad app list --display-name "AntRunner" --query "[].appId" -o tsv
```
Copy the app ID from the output.

Now, use the following command to add the `User.Read` permission:
```sh
az ad app permission add --id <your-app-id> --api 00000003-0000-0000-c000-000000000000 --api-permissions 311a71cc-e848-46a1-bdf8-97ff7156d8e6=Scope
```
Replace `<your-app-id>` with the app ID you copied earlier. The GUID `00000003-0000-0000-c000-000000000000` represents the Microsoft Graph API, and `311a71cc-e848-46a1-bdf8-97ff7156d8e6` represents the `User.Read` permission.

### Step 5: Enable Public Client Flows
Use the following command to enable public client flows:
```sh
az ad app update --id <your-app-id> --set publicClient=true
```
Replace `<your-app-id>` with the app ID you copied earlier.

### Step 6: Note Down Application (Client) ID and Directory (Tenant) ID
Use the following command to get the details of your app registration, including the client ID and tenant ID:
```sh
az ad app show --id <your-app-id>
```
Note down the `appId` (Application (client) ID) and `publisherDomain` (Directory (tenant) ID).

For more detailed guidance, refer to the official Microsoft documentation:
- [Register an application with the Microsoft identity platform](https://learn.microsoft.com/en-us/graph/auth-register-app-v2)
- [Sign in with Azure CLI at a command line](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

These steps will help you set up an app registration for interactive device authentication using both the Azure Portal and Azure CLI, and configure the necessary permissions to read the user's profile from Microsoft Graph. 

## First steps
- First, download AntRunLib from Nuget
- At least once, setup the enviroment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.1

## Get an access token to read your profile with the Application (Client) ID and Directory (Tenant) ID you created above

In [14]:
string[] scopes = ["User.Read"];
var oAuthToken = await OAuthHelper.GetToken("6ec70645-4a94-4d41-abfa-fdd579a6076d", "a5ef985b-4e9a-46cc-854d-4db49be6e9df", scopes);

## Ensure the assistant is created

In [15]:
var assistantId = await AssistantUtility.Create("MsGraphUserProfile", config);
Console.WriteLine(assistantId)

asst_Kc3PEljgaTXr6ouY64GihiK0


## Run it!

In [16]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "MsGraphUserProfile",
    Instructions = "What is my name?",
    OauthUserAccessToken = oAuthToken,
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: What is my name?

Assistant: I called the tool named UserProfileAnt with these arguments:
```
{}
```
and got this result:
```
{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users/$entity","businessPhones":[],"displayName":"Doug Ware","givenName":"Doug","jobTitle":null,"mail":"dougware@elumenotion.com","mobilePhone":null,"officeLocation":null,"preferredLanguage":"en-US","surname":"Ware","userPrincipalName":"dougware@elumenotion.com","id":"d30ce525-7d0a-4672-bc8a-cef86a550879"}
```

Your name is Doug Ware.


In [12]:
output.LastMessage.DisplayAs("text/markdown");

Your name is Doug Ware.

## Clean up

In [17]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "MsGraphUserProfile");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
